In [ ]:
import pandas as pd
import sqlite3

In [ ]:
# Ruta del archivo
archivo = '/content/drive/MyDrive/Prueba_tecnica_tuya/rachas.xlsx'

# Leer las hojas
historia = pd.read_excel(archivo, sheet_name='historia')
retiros = pd.read_excel(archivo, sheet_name='retiros')


In [ ]:
# Crear base de datos SQLite
conn = sqlite3.connect('rachas.db')

# Subir a la base de datos
historia.to_sql('historia', conn, if_exists='replace', index=False)
retiros.to_sql('retiros', conn, if_exists='replace', index=False)

5

In [ ]:
#Clasificar los niveles de saldo (vista historia_nivel)
query_niveles = """
DROP VIEW IF EXISTS historia_nivel;
CREATE VIEW historia_nivel AS
SELECT *,
  CASE
    WHEN saldo >= 0         AND saldo < 300000   THEN 'N0'
    WHEN saldo >= 300000    AND saldo < 1000000  THEN 'N1'
    WHEN saldo >= 1000000   AND saldo < 3000000  THEN 'N2'
    WHEN saldo >= 3000000   AND saldo < 5000000  THEN 'N3'
    WHEN saldo >= 5000000                        THEN 'N4'
  END AS nivel
FROM historia;
"""
conn.executescript(query_niveles)

In [ ]:
# Crear calendario de meses
min_date = pd.to_datetime(historia["corte_mes"].min())
max_date = pd.to_datetime(historia["corte_mes"].max())
rango_fechas = pd.date_range(min_date, max_date, freq='MS')
calendario = pd.DataFrame({'corte_mes': rango_fechas})

# Crear combinaciones cliente-fecha
clientes = pd.DataFrame(historia["identificacion"].unique(), columns=["identificacion"])
clientes["key"] = 1
calendario["key"] = 1
completo = clientes.merge(calendario, on="key")[["identificacion", "corte_mes"]]

# Subir tablas auxiliares
calendario.to_sql("calendario", conn, if_exists="replace", index=False)
completo.to_sql("completo", conn, if_exists="replace", index=False)

3450

In [ ]:
#Clasificar saldos por nivel con control de N0 y retiros
query_union = """
DROP VIEW IF EXISTS historia_completa;
CREATE VIEW historia_completa AS
SELECT
  c.identificacion,
  c.corte_mes,
  COALESCE(h.saldo, 0) AS saldo,
  r.fecha_retiro,
  CASE
    WHEN r.fecha_retiro IS NOT NULL AND c.corte_mes > r.fecha_retiro THEN NULL
    ELSE COALESCE(
      CASE
        WHEN h.saldo >= 0 AND h.saldo < 300000 THEN 'N0'
        WHEN h.saldo >= 300000 AND h.saldo < 1000000 THEN 'N1'
        WHEN h.saldo >= 1000000 AND h.saldo < 3000000 THEN 'N2'
        WHEN h.saldo >= 3000000 AND h.saldo < 5000000 THEN 'N3'
        WHEN h.saldo >= 5000000 THEN 'N4'
        ELSE 'N0'
      END, 'N0'
    )
  END AS nivel
FROM completo c
LEFT JOIN historia h
  ON c.identificacion = h.identificacion AND c.corte_mes = h.corte_mes
LEFT JOIN retiros r
  ON c.identificacion = r.identificacion;
"""
conn.executescript(query_union)


In [ ]:
#Detectar rachas por nivel
query_rachas = """
DROP VIEW IF EXISTS rachas_consecutivas;
CREATE VIEW rachas_consecutivas AS
WITH numerada AS (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY identificacion ORDER BY corte_mes) -
         ROW_NUMBER() OVER (PARTITION BY identificacion, nivel ORDER BY corte_mes) AS grupo
  FROM historia_completa
  WHERE nivel IS NOT NULL
),
agrupada AS (
  SELECT identificacion, nivel,
         MIN(corte_mes) AS fecha_inicio,
         MAX(corte_mes) AS fecha_fin,
         COUNT(*) AS racha
  FROM numerada
  GROUP BY identificacion, nivel, grupo
)
SELECT * FROM agrupada;
"""
conn.executescript(query_rachas)


In [ ]:
#Consulta final con parámetros
# Parámetros
fecha_base = "2024-12-01"  # Puedes cambiarlo
racha_min = 3

query_final = f"""
WITH filtrado AS (
  SELECT *
  FROM rachas_consecutivas
  WHERE racha >= {racha_min}
    AND fecha_fin <= DATE('{fecha_base}')
),
ranking AS (
  SELECT *,
         RANK() OVER (PARTITION BY identificacion ORDER BY racha DESC, fecha_fin DESC) AS rk
  FROM filtrado
)
SELECT identificacion, racha, fecha_fin, nivel
FROM ranking
WHERE rk = 1;
"""

# Ejecutar y mostrar resultado
df_resultado = pd.read_sql(query_final, conn)
df_resultado


,identificacion,racha,fecha_fin,nivel
0,0TTW5R9RRCJ0A9E5F,21,2024-10-01 00:00:00,N0
1,THJ9OBJH3W6ANRCMS,12,2024-01-01 00:00:00,N0


In [ ]:
readme_content = """# Prueba Técnica - Tuya

## Punto 3: Rachas de Niveles de Deuda

Este módulo analiza la historia de saldos mensuales de clientes y detecta rachas consecutivas donde los clientes se mantienen en el mismo nivel de deuda.

### 📌 Objetivo

- Clasificar los saldos en niveles de deuda.
- Identificar secuencias consecutivas (rachas) de un mismo nivel.
- Seleccionar la racha más larga (o más reciente en caso de empate) por cliente.

### 📂 Archivos de entrada

- `rachas.xlsx` con dos hojas:
  - `historia`: contiene identificación, corte_mes y saldo.
  - `retiros`: contiene la fecha de retiro del cliente.

### 🛠️ Herramientas utilizadas

- Python 3
- pandas
- sqlite3
- SQL (para lógica de procesamiento de rachas)

### 🧮 Niveles de saldo

- `N0`: saldo entre 0 y <300,000
- `N1`: 300,000 <= saldo < 1,000,000
- `N2`: 1,000,000 <= saldo < 3,000,000
- `N3`: 3,000,000 <= saldo < 5,000,000
- `N4`: saldo >= 5,000,000

### 📋 Instrucciones

1. Asegúrate de tener el archivo `rachas.xlsx` en la ruta correspondiente.
2. Ejecuta el script en Google Colab o entorno local.
3. Se generará una base de datos SQLite (`rachas.db`) y múltiples vistas auxiliares para facilitar el análisis.
4. El resultado final será una tabla con la racha más larga por cliente.

### Archivos generados

- `rachas.db`: base de datos SQLite con todas las tablas y vistas necesarias.
- DataFrame con el resultado final del análisis (puede exportarse a CSV si se desea).

"""

# Guardar README
ruta_readme = "/content/drive/MyDrive/Prueba_tecnica_tuya/Ejercicio_3_rachas/README.md"
with open(ruta_readme, "w", encoding="utf-8") as f:
    f.write(readme_content)

print(f"README.md creado en: {ruta_readme}")
